In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_script_2025_04_04_17_27.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.005372
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,0.0,0.000000
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,0.0,0.000000


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.005372,0.005372,0.000029,1.0,0.005372
1,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
2,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...
116046,graph_powerlaw_cluster_graph_n9,2.0,2.289660,0.289660,0.083903,0.0,0.144830
116047,graph_powerlaw_cluster_graph_n9,1.0,0.914653,0.085347,0.007284,0.0,0.085347
116048,graph_powerlaw_cluster_graph_n9,3.0,3.213434,0.213434,0.045554,0.0,0.071145
116049,graph_powerlaw_cluster_graph_n9,4.0,3.765309,0.234691,0.055080,0.0,0.058673


In [9]:
df['RE'].sum()/len(df)

0.06231106748705521

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,28168,2222.984285,0.078919,0.094043
1,2.0,27357,1683.763943,0.061548,0.028029
2,4.0,17623,980.273423,0.055625,0.090081
3,0.0,17593,619.534066,0.035215,0.003519
4,1.0,16721,1206.947972,0.072182,0.008048
5,5.0,6078,326.280261,0.053682,0.149203
6,6.0,1347,106.450404,0.079028,0.433590
7,7.0,479,39.495885,0.082455,0.502060
8,8.0,347,15.331682,0.044184,0.154348
9,9.0,202,11.410880,0.056490,0.334682


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n9,69120,3921.313661,0.056732,0.045816
1,graph_random_regular_graph_n8_d4,19532,1467.544186,0.075135,0.079670
2,star_graph_n15,12288,722.747196,0.058817,0.135497
3,graph_powerlaw_cluster_graph_n8,7168,531.913998,0.074207,0.052564
4,graph_random_regular_graph_n7_d4,3907,345.853274,0.088521,0.071559
5,star_graph_n13,2663,126.767909,0.047603,0.098398
6,graph_powerlaw_cluster_graph_n7,1350,114.281964,0.084653,0.045662
7,star_graph_n7,23,0.839504,0.036500,0.115885


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n9,65863,3900.929648,0.059228,0.048074
1,graph_random_regular_graph_n8_d4,19145,1465.660881,0.076556,0.081278
2,graph_powerlaw_cluster_graph_n8,6422,526.817994,0.082033,0.058648
3,graph_random_regular_graph_n7_d4,3779,345.061969,0.091310,0.073976
4,star_graph_n15,1607,205.753646,0.128036,1.001892
5,graph_powerlaw_cluster_graph_n7,1248,113.459007,0.090913,0.049373
6,star_graph_n13,391,53.215826,0.136102,0.654253
7,star_graph_n7,3,0.828655,0.276218,0.888434


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,425,35.295832,0.083049,0.040769
1,graph_powerlaw_cluster_graph_n7,1.0,379,47.666371,0.125769,0.022032
2,graph_powerlaw_cluster_graph_n7,3.0,307,22.830196,0.074365,0.082510
3,graph_powerlaw_cluster_graph_n7,4.0,121,6.694948,0.055330,0.073607
4,graph_powerlaw_cluster_graph_n7,0.0,102,0.822957,0.008068,0.000255
...,...,...,...,...,...,...
58,star_graph_n15,12.0,3,0.672811,0.224270,7.271605
59,star_graph_n15,13.0,1,0.322047,0.322047,17.527705
60,star_graph_n7,0.0,20,0.010849,0.000542,0.000003
61,star_graph_n7,3.0,2,0.615375,0.307688,0.968744
